<a href="https://colab.research.google.com/github/MLDreamer/AIMathematicallyexplained/blob/main/Geometry_of_Attention_Mechanism_by_Linear_Algebra.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation, PillowWriter
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import matplotlib.patches as mpatches
from matplotlib.patches import FancyArrowPatch
from mpl_toolkits.mplot3d import proj3d

plt.style.use('seaborn-v0_8-whitegrid')

class Arrow3D(FancyArrowPatch):
    def __init__(self, xs, ys, zs, *args, **kwargs):
        super().__init__((0,0), (0,0), *args, **kwargs)
        self._verts3d = xs, ys, zs

    def do_3d_projection(self, renderer=None):
        xs3d, ys3d, zs3d = self._verts3d
        xs, ys, zs = proj3d.proj_transform(xs3d, ys3d, zs3d, self.axes.M)
        self.set_positions((xs[0],ys[0]),(xs[1],ys[1]))
        return np.min(zs)

class RomanceAttentionGeometry:
    """
    Complete attention mechanism with romance analogy
    Following rigorous linear algebra from the article
    """
    def __init__(self):
        # Personality vectors in 3D: [humor, intelligence, kindness]
        self.X = np.array([
            [0.9, 0.8, 0.7],   # Alice
            [0.5, 0.95, 0.6],  # Bob
            [0.2, 0.6, 0.8]    # Carol
        ])
        self.people = ['Alice', 'Bob', 'Carol']
        self.colors = ['#FF69B4', '#4169E1', '#32CD32']  # Pink, Blue, Green
        self.you = np.array([0.85, 0.90, 0.75])

        # Weight matrices (matching article exactly)
        self.Wq = np.array([
            [0.8, 0.2],
            [0.3, 0.9],
            [0.1, 0.5]
        ])  # 3D -> 2D query projection

        self.Wk = np.array([
            [0.7, 0.3],
            [0.4, 0.8],
            [0.2, 0.6]
        ])  # 3D -> 2D key projection

        self.Wv = np.eye(3)  # Identity (values = original personality)

        self.compute_all()

    def compute_all(self):
        """Compute all intermediate values"""
        # Query: what YOU are looking for (2D)
        self.q = self.you @ self.Wq  # [1.025, 1.355]

        # Keys: how others present themselves (n×2)
        self.K = self.X @ self.Wk
        # k_Alice = [1.09, 1.33]
        # k_Bob = [0.85, 1.27]
        # k_Carol = [0.54, 1.02]

        # Values: what others actually offer (n×3)
        self.V = self.X @ self.Wv  # = X since Wv = I

        # Compatibility scores (dot products)
        self.scores = self.q @ self.K.T  # [2.919, 2.592, 1.936]

        # Scaled scores
        d_k = self.K.shape[1]
        self.scaled_scores = self.scores / np.sqrt(d_k)  # [2.064, 1.833, 1.369]

        # Attention weights (softmax)
        exp_scores = np.exp(self.scaled_scores)
        self.attention_weights = exp_scores / np.sum(exp_scores)  # [0.436, 0.346, 0.218]

        # Output: weighted average of values
        self.output = self.attention_weights @ self.V  # [0.609, 0.809, 0.687]

        # Compute angles for visualization
        self.compute_angles()

    def compute_angles(self):
        """Compute angles between vectors for geometric interpretation"""
        self.angles = {}
        for i, person in enumerate(self.people):
            vec = self.X[i]
            norm_you = np.linalg.norm(self.you)
            norm_vec = np.linalg.norm(vec)
            cos_theta = np.dot(self.you, vec) / (norm_you * norm_vec)
            theta = np.arccos(np.clip(cos_theta, -1, 1))
            self.angles[person] = np.degrees(theta)


def gif_1_personality_space_3d():
    """
    GIF 1: Three people appearing in 3D personality space
    Shows [humor, intelligence, kindness] as axes
    People as vectors from origin with human silhouettes
    """
    viz = RomanceAttentionGeometry()

    fig = plt.figure(figsize=(14, 12))
    ax = fig.add_subplot(111, projection='3d')

    def animate(frame):
        ax.clear()

        # Rotate view slowly
        angle = 30 + (frame % 120) * 0.5
        ax.view_init(elev=25, azim=angle)

        ax.set_xlim(0, 1.0)
        ax.set_ylim(0, 1.0)
        ax.set_zlim(0, 1.0)
        ax.set_xlabel('Humor →', fontsize=13, fontweight='bold', labelpad=10)
        ax.set_ylabel('Intelligence →', fontsize=13, fontweight='bold', labelpad=10)
        ax.set_zlabel('Kindness →', fontsize=13, fontweight='bold', labelpad=10)
        ax.set_title('Personality Space: People as Vectors in ℝ³\n(Each person = point in 3D personality space)',
                    fontsize=14, fontweight='bold', pad=20)

        # Grid
        ax.grid(True, alpha=0.3, linewidth=1)

        # Origin
        ax.scatter(0, 0, 0, color='black', s=100, marker='o', zorder=5)

        # People appear sequentially
        num_people = min(frame // 30 + 1, 3)

        for i in range(num_people):
            alpha = min((frame - i*30) / 30, 1.0) if frame >= i*30 else 0

            if alpha > 0:
                x, y, z = viz.X[i]

                # Arrow from origin to person
                arrow = Arrow3D([0, x], [0, y], [0, z],
                               mutation_scale=20, lw=3, arrowstyle='-|>',
                               color=viz.colors[i], alpha=alpha*0.6)
                ax.add_artist(arrow)

                # Person as sphere
                ax.scatter(x, y, z, color=viz.colors[i], s=800,
                          edgecolors='black', linewidths=3, alpha=alpha, zorder=10)

                # Label with coordinates
                if alpha > 0.5:
                    ax.text(x+0.08, y+0.05, z, f'{viz.people[i]}\n[{x:.1f}, {y:.1f}, {z:.1f}]',
                           fontsize=11, fontweight='bold', color=viz.colors[i],
                           bbox=dict(boxstyle='round', facecolor='white', alpha=0.9, edgecolor=viz.colors[i], linewidth=2))

        # Add explanation after all appear
        if frame > 90:
            ax.text2D(0.5, 0.02, 'Similar personalities → nearby points | Distance measures dissimilarity',
                     transform=ax.transAxes, fontsize=10, ha='center', style='italic',
                     bbox=dict(boxstyle='round', facecolor='yellow', alpha=0.7))

    anim = FuncAnimation(fig, animate, frames=150, interval=100)
    anim.save('gif1_personality_space_3d.gif', writer=PillowWriter(fps=10))
    plt.close()
    print("✓ GIF 1: 3D Personality space (slow rotating, people as vectors)")


def gif_2_inner_product_angle():
    """
    GIF 2: Showing angle between You and Alice
    Visualizes inner product = ||x|| ||y|| cos(θ)
    Shows arc of angle θ and computes compatibility
    """
    viz = RomanceAttentionGeometry()

    fig = plt.figure(figsize=(14, 12))
    ax = fig.add_subplot(111, projection='3d')

    def animate(frame):
        ax.clear()

        angle = 20 + (frame % 100) * 0.6
        ax.view_init(elev=20, azim=angle)

        ax.set_xlim(0, 1.0)
        ax.set_ylim(0, 1.0)
        ax.set_zlim(0, 1.0)
        ax.set_xlabel('Humor', fontsize=12, fontweight='bold')
        ax.set_ylabel('Intelligence', fontsize=12, fontweight='bold')
        ax.set_zlabel('Kindness', fontsize=12, fontweight='bold')
        ax.set_title('Inner Product as Compatibility: ⟨x,y⟩ = ||x|| ||y|| cos(θ)\n(Angle θ measures alignment)',
                    fontsize=13, fontweight='bold', pad=20)

        ax.grid(True, alpha=0.3)

        # Your vector
        x_you, y_you, z_you = viz.you
        arrow_you = Arrow3D([0, x_you], [0, y_you], [0, z_you],
                           mutation_scale=20, lw=4, arrowstyle='-|>',
                           color='purple', alpha=0.9)
        ax.add_artist(arrow_you)
        ax.scatter(x_you, y_you, z_you, color='purple', s=600, marker='*',
                  edgecolors='black', linewidths=3, zorder=10)
        ax.text(x_you+0.08, y_you, z_you+0.05, 'YOU',
               fontsize=12, fontweight='bold', color='purple',
               bbox=dict(boxstyle='round', facecolor='white', alpha=0.9))

        # Focus on Alice
        progress = min(frame / 60, 1.0)

        if progress > 0:
            x_a, y_a, z_a = viz.X[0]

            # Alice's vector
            arrow_alice = Arrow3D([0, x_a], [0, y_a], [0, z_a],
                                 mutation_scale=20, lw=4, arrowstyle='-|>',
                                 color=viz.colors[0], alpha=progress)
            ax.add_artist(arrow_alice)
            ax.scatter(x_a, y_a, z_a, color=viz.colors[0], s=600,
                      edgecolors='black', linewidths=3, alpha=progress, zorder=10)

            if progress > 0.3:
                ax.text(x_a+0.08, y_a, z_a+0.05, 'Alice',
                       fontsize=12, fontweight='bold', color=viz.colors[0],
                       bbox=dict(boxstyle='round', facecolor='white', alpha=0.9))

            # Show angle arc
            if progress > 0.6:
                # Compute angle
                dot_product = np.dot(viz.you, viz.X[0])
                norm_you = np.linalg.norm(viz.you)
                norm_alice = np.linalg.norm(viz.X[0])
                cos_theta = dot_product / (norm_you * norm_alice)
                theta_rad = np.arccos(np.clip(cos_theta, -1, 1))
                theta_deg = np.degrees(theta_rad)

                # Draw angle annotation
                ax.text2D(0.5, 0.95, f'⟨YOU, Alice⟩ = {dot_product:.3f}',
                         transform=ax.transAxes, fontsize=11, ha='center', fontweight='bold',
                         bbox=dict(boxstyle='round', facecolor='lightblue', alpha=0.8))

                ax.text2D(0.5, 0.90, f'||YOU|| = {norm_you:.3f}, ||Alice|| = {norm_alice:.3f}',
                         transform=ax.transAxes, fontsize=10, ha='center',
                         bbox=dict(boxstyle='round', facecolor='lightgreen', alpha=0.7))

                ax.text2D(0.5, 0.85, f'cos(θ) = {cos_theta:.3f}, θ = {theta_deg:.1f}°',
                         transform=ax.transAxes, fontsize=10, ha='center', fontweight='bold',
                         bbox=dict(boxstyle='round', facecolor='yellow', alpha=0.8))

                ax.text2D(0.5, 0.05, f'Small angle → High alignment → HIGH COMPATIBILITY!',
                         transform=ax.transAxes, fontsize=11, ha='center', style='italic',
                         bbox=dict(boxstyle='round', facecolor='pink', alpha=0.7))

    anim = FuncAnimation(fig, animate, frames=120, interval=100)
    anim.save('gif2_inner_product_angle.gif', writer=PillowWriter(fps=10))
    plt.close()
    print("✓ GIF 2: Inner product geometry (angle θ, compatibility)")


def gif_3_query_projection_3d_to_2d():
    """
    GIF 3: Your 3D preference transforming to 2D query via W_Q
    Shows matrix multiplication geometrically as projection
    """
    viz = RomanceAttentionGeometry()

    fig = plt.figure(figsize=(16, 12))
    ax = fig.add_subplot(111, projection='3d')

    def animate(frame):
        ax.clear()

        angle = 30 + (frame % 120) * 0.5
        ax.view_init(elev=25, azim=angle)

        ax.set_xlim(0, 1.2)
        ax.set_ylim(0, 1.2)
        ax.set_zlim(0, 1.5)
        ax.set_xlabel('Dimension 1', fontsize=12, fontweight='bold')
        ax.set_ylabel('Dimension 2', fontsize=12, fontweight='bold')
        ax.set_zlabel('Dimension 3', fontsize=12, fontweight='bold')
        ax.set_title('Query Projection: x_you ∈ ℝ³ → q ∈ ℝ² via W_Q\n(Linear transformation to "what I seek" subspace)',
                    fontsize=13, fontweight='bold', pad=20)

        ax.grid(True, alpha=0.3)

        # Your original 3D vector
        x, y, z = viz.you
        arrow_orig = Arrow3D([0, x], [0, y], [0, z],
                            mutation_scale=20, lw=4, arrowstyle='-|>',
                            color='purple', alpha=0.7)
        ax.add_artist(arrow_orig)
        ax.scatter(x, y, z, color='purple', s=500, marker='*',
                  edgecolors='black', linewidths=3, alpha=0.7)
        ax.text(x+0.08, y, z+0.08, 'x_you (3D)\n[0.85, 0.90, 0.75]',
               fontsize=10, color='purple', fontweight='bold',
               bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

        # Draw 2D query plane (z=0 plane, but elevated for visibility)
        progress = min(frame / 80, 1.0)

        if progress > 0.2:
            # Show the 2D query plane
            xx, yy = np.meshgrid(np.linspace(0, 1.2, 10), np.linspace(0, 1.5, 10))
            zz = np.zeros_like(xx)
            ax.plot_surface(xx, yy, zz, alpha=0.2, color='cyan')

            ax.text2D(0.3, 0.9, '2D Query Subspace (col(W_Q))',
                     transform=ax.transAxes, fontsize=11, fontweight='bold',
                     bbox=dict(boxstyle='round', facecolor='cyan', alpha=0.6))

        if progress > 0.4:
            fade = min((progress - 0.4) / 0.4, 1.0)

            # Project to 2D query (q lives in xy-plane, z=0)
            q_x, q_y = viz.q
            q_z = 0

            # Morphing arrow
            current_z = z * (1 - fade) + q_z * fade
            current_x = x * (1 - fade) + q_x * fade
            current_y = y * (1 - fade) + q_y * fade

            arrow_query = Arrow3D([0, current_x], [0, current_y], [0, current_z],
                                 mutation_scale=20, lw=5, arrowstyle='-|>',
                                 color='darkviolet', alpha=fade)
            ax.add_artist(arrow_query)

            ax.scatter(current_x, current_y, current_z, color='darkviolet', s=600,
                      edgecolors='black', linewidths=3, alpha=fade, zorder=10)

            if fade > 0.7:
                ax.text(q_x+0.1, q_y+0.1, 0.05, f'q (2D query)\n[{q_x:.2f}, {q_y:.2f}]',
                       fontsize=11, color='darkviolet', fontweight='bold',
                       bbox=dict(boxstyle='round', facecolor='yellow', alpha=0.9))

                # Show transformation
                ax.text2D(0.5, 0.05, f'Linear transformation: q = x_you × W_Q\n' +
                         f'Projection from 3D personality → 2D "what I seek"',
                         transform=ax.transAxes, fontsize=10, ha='center',
                         bbox=dict(boxstyle='round', facecolor='lightgreen', alpha=0.8))

    anim = FuncAnimation(fig, animate, frames=140, interval=100)
    anim.save('gif3_query_projection_3d_to_2d.gif', writer=PillowWriter(fps=10))
    plt.close()
    print("✓ GIF 3: Query projection 3D→2D (geometric transformation)")


def gif_4_key_projections_all():
    """
    GIF 4: All three people's 3D personalities projecting to 2D keys
    Shows W_K transformation geometrically
    """
    viz = RomanceAttentionGeometry()

    fig = plt.figure(figsize=(16, 12))
    ax = fig.add_subplot(111, projection='3d')

    def animate(frame):
        ax.clear()

        angle = 40 + (frame % 150) * 0.4
        ax.view_init(elev=30, azim=angle)

        ax.set_xlim(0, 1.2)
        ax.set_ylim(0, 1.2)
        ax.set_zlim(0, 1.5)
        ax.set_xlabel('Dimension 1', fontsize=12, fontweight='bold')
        ax.set_ylabel('Dimension 2', fontsize=12, fontweight='bold')
        ax.set_zlabel('Dimension 3 → 0', fontsize=12, fontweight='bold')
        ax.set_title('Key Projections: X ∈ ℝ³ˣ³ → K ∈ ℝ³ˣ² via W_K\n(How people present their "dating profiles")',
                    fontsize=13, fontweight='bold', pad=20)

        ax.grid(True, alpha=0.3)

        # Draw 2D key plane
        xx, yy = np.meshgrid(np.linspace(0, 1.2, 10), np.linspace(0, 1.5, 10))
        zz = np.zeros_like(xx)
        ax.plot_surface(xx, yy, zz, alpha=0.15, color='lightblue')

        # Process each person sequentially
        phase = frame // 50
        progress = (frame % 50) / 50

        for i in range(3):
            x, y, z = viz.X[i]

            # Original 3D personality (faded)
            arrow_3d = Arrow3D([0, x], [0, y], [0, z],
                              mutation_scale=15, lw=2, arrowstyle='-|>',
                              color=viz.colors[i], alpha=0.3, linestyle='--')
            ax.add_artist(arrow_3d)
            ax.scatter(x, y, z, color=viz.colors[i], s=300,
                      alpha=0.3, edgecolors='black', linewidths=2)

            if i <= phase:
                alpha = progress if i == phase else 1.0

                # Project to 2D key
                k_x, k_y = viz.K[i]
                k_z = 0

                if i < phase:
                    alpha = 1.0
                    current_x, current_y, current_z = k_x, k_y, k_z
                else:
                    current_x = x * (1 - alpha) + k_x * alpha
                    current_y = y * (1 - alpha) + k_y * alpha
                    current_z = z * (1 - alpha) + k_z * alpha

                arrow_key = Arrow3D([0, current_x], [0, current_y], [0, current_z],
                                   mutation_scale=20, lw=4, arrowstyle='-|>',
                                   color=viz.colors[i], alpha=alpha)
                ax.add_artist(arrow_key)

                ax.scatter(current_x, current_y, current_z, color=viz.colors[i], s=500,
                          edgecolors='black', linewidths=3, alpha=alpha, zorder=10)

                if alpha > 0.6:
                    ax.text(current_x+0.08, current_y, current_z+0.05,
                           f'k_{viz.people[i]}\n[{k_x:.2f}, {k_y:.2f}]',
                           fontsize=10, color=viz.colors[i], fontweight='bold',
                           bbox=dict(boxstyle='round', facecolor='white', alpha=0.9))

        ax.text2D(0.5, 0.95, '2D Key Subspace: How they present themselves',
                 transform=ax.transAxes, fontsize=11, ha='center', fontweight='bold',
                 bbox=dict(boxstyle='round', facecolor='lightblue', alpha=0.7))

        ax.text2D(0.5, 0.05, 'K = X × W_K (Linear projection to dating profile space)',
                 transform=ax.transAxes, fontsize=10, ha='center',
                 bbox=dict(boxstyle='round', facecolor='yellow', alpha=0.7))

    anim = FuncAnimation(fig, animate, frames=180, interval=100)
    anim.save('gif4_key_projections_all.gif', writer=PillowWriter(fps=10))
    plt.close()
    print("✓ GIF 4: Key projections 3D→2D (all three people)")


def gif_5_dot_product_scores_3d():
    """
    GIF 5: Computing dot products q·k_i in 2D plane
    Shows scores as vertical bars rising from base
    """
    viz = RomanceAttentionGeometry()

    fig = plt.figure(figsize=(14, 12))
    ax = fig.add_subplot(111, projection='3d')

    def animate(frame):
        ax.clear()

        angle = 30 + (frame % 150) * 0.5
        ax.view_init(elev=25, azim=angle)

        ax.set_xlim(0, 1.3)
        ax.set_ylim(0, 1.5)
        ax.set_zlim(0, 3.5)
        ax.set_xlabel('Query/Key Dim 1', fontsize=11, fontweight='bold')
        ax.set_ylabel('Query/Key Dim 2', fontsize=11, fontweight='bold')
        ax.set_zlabel('Compatibility Score\n(Dot Product)', fontsize=11, fontweight='bold')
        ax.set_title('Computing Compatibility: score_i = q · k_i = ⟨q, k_i⟩\n(Geometric similarity in 2D key/query space)',
                    fontsize=13, fontweight='bold', pad=20)

        ax.grid(True, alpha=0.3)

        # 2D base plane
        xx, yy = np.meshgrid(np.linspace(0, 1.3, 10), np.linspace(0, 1.5, 10))
        zz = np.zeros_like(xx)
        ax.plot_surface(xx, yy, zz, alpha=0.1, color='lightgray')

        # Your query in 2D
        q_x, q_y = viz.q
        arrow_q = Arrow3D([0, q_x], [0, q_y], [0, 0],
                         mutation_scale=20, lw=5, arrowstyle='-|>',
                         color='purple', alpha=0.9)
        ax.add_artist(arrow_q)
        ax.scatter(q_x, q_y, 0, color='purple', s=600, marker='*',
                  edgecolors='black', linewidths=3, zorder=10)
        ax.text(q_x+0.1, q_y, 0.1, 'q (YOU)',
               fontsize=11, color='purple', fontweight='bold',
               bbox=dict(boxstyle='round', facecolor='white', alpha=0.9))

        # Keys in 2D
        for i in range(3):
            k_x, k_y = viz.K[i]
            arrow_k = Arrow3D([0, k_x], [0, k_y], [0, 0],
                             mutation_scale=15, lw=3, arrowstyle='-|>',
                             color=viz.colors[i], alpha=0.5, linestyle='--')
            ax.add_artist(arrow_k)
            ax.scatter(k_x, k_y, 0, color=viz.colors[i], s=400,
                      alpha=0.6, edgecolors='black', linewidths=2)
            ax.text(k_x+0.05, k_y, -0.05, f'k_{viz.people[i]}',
                   fontsize=9, color=viz.colors[i], fontweight='bold')

        # Compute and show scores sequentially
        phase = (frame // 50) % 4

        if phase > 0:
            for i in range(min(phase, 3)):
                progress = min((frame - (i+1)*50) / 50, 1.0) if frame >= (i+1)*50 else 0

                if progress > 0:
                    k_x, k_y = viz.K[i]
                    score = viz.scores[i]
                    current_height = score * progress

                    # Vertical line showing score
                    ax.plot([k_x, k_x], [k_y, k_y], [0, current_height],
                           color=viz.colors[i], linewidth=5, alpha=0.8)

                    # Score sphere at top
                    ax.scatter(k_x, k_y, current_height, color=viz.colors[i], s=400,
                              edgecolors='black', linewidths=3, alpha=0.9, zorder=10)

                    if progress > 0.6:
                        ax.text(k_x+0.08, k_y, current_height+0.1,
                               f'{viz.people[i]}\nscore={score:.3f}',
                               fontsize=10, fontweight='bold', color=viz.colors[i],
                               bbox=dict(boxstyle='round', facecolor='white', alpha=0.9))

                        # Show calculation
                        if i == phase - 1 and progress > 0.8:
                            ax.text2D(0.5, 0.95, f'q · k_{viz.people[i]} = [{viz.q[0]:.2f}]·[{viz.K[i,0]:.2f}] + [{viz.q[1]:.2f}]·[{viz.K[i,1]:.2f}] = {score:.3f}',
                                     transform=ax.transAxes, fontsize=10, ha='center',
                                     bbox=dict(boxstyle='round', facecolor='yellow', alpha=0.8))

        ax.text2D(0.5, 0.05, 'Higher bar = Higher compatibility!',
                 transform=ax.transAxes, fontsize=11, ha='center', style='italic',
                 bbox=dict(boxstyle='round', facecolor='lightgreen', alpha=0.7))

    anim = FuncAnimation(fig, animate, frames=200, interval=100)
    anim.save('gif5_dot_product_scores_3d.gif', writer=PillowWriter(fps=10))
    plt.close()
    print("✓ GIF 5: Dot product compatibility scores (3D vertical bars)")


def gif_6_softmax_transformation():
    """
    GIF 6: Softmax transforming raw scores to probability distribution
    Shows bars morphing from scores to percentages
    """
    viz = RomanceAttentionGeometry()

    fig = plt.figure(figsize=(16, 10))
    ax1 = fig.add_subplot(131, projection='3d')
    ax2 = fig.add_subplot(132, projection='3d')
    ax3 = fig.add_subplot(133, projection='3d')

    def animate(frame):
        for ax in [ax1, ax2, ax3]:
            ax.clear()
            ax.set_xlim(0, 3)
            ax.set_ylim(0, 1)
            ax.set_zlim(0, 3.5)
            ax.view_init(elev=20, azim=30)
            ax.grid(True, alpha=0.3)

        ax1.set_title('Raw Scores\n(Dot Products)', fontsize=11, fontweight='bold')
        ax2.set_title('Scaled Scores\n(÷ √d_k)', fontsize=11, fontweight='bold')
        ax3.set_title('Attention Weights\n(Softmax)', fontsize=11, fontweight='bold')

        ax1.set_zlabel('Score', fontsize=10)
        ax2.set_zlabel('Scaled', fontsize=10)
        ax3.set_zlabel('Weight', fontsize=10)

        # Raw scores
        for i in range(3):
            x_pos = i
            score = viz.scores[i]
            ax1.bar3d(x_pos, 0, 0, 0.6, 0.6, score,
                     color=viz.colors[i], alpha=0.7, edgecolor='black', linewidth=2)
            ax1.text(x_pos+0.3, 0.3, score+0.1, f'{score:.2f}',
                    ha='center', fontsize=10, fontweight='bold')

        # Scaled scores
        progress1 = min(frame / 60, 1.0)
        for i in range(3):
            x_pos = i
            raw = viz.scores[i]
            scaled = viz.scaled_scores[i]
            current = raw * (1 - progress1) + scaled * progress1

            ax2.bar3d(x_pos, 0, 0, 0.6, 0.6, current,
                     color=viz.colors[i], alpha=0.7, edgecolor='black', linewidth=2)
            if progress1 > 0.5:
                ax2.text(x_pos+0.3, 0.3, current+0.1, f'{scaled:.2f}',
                        ha='center', fontsize=10, fontweight='bold')

        # Attention weights (after softmax)
        if frame > 60:
            progress2 = min((frame - 60) / 60, 1.0)
            for i in range(3):
                x_pos = i
                scaled = viz.scaled_scores[i]
                weight = viz.attention_weights[i]

                # Morph from scaled to weight (different scale)
                current = scaled * (1 - progress2) + weight * 3 * progress2

                ax3.bar3d(x_pos, 0, 0, 0.6, 0.6, weight * 3,
                         color=viz.colors[i], alpha=0.7, edgecolor='black', linewidth=2)

                if progress2 > 0.5:
                    ax3.text(x_pos+0.3, 0.3, weight*3+0.1, f'{weight:.1%}',
                            ha='center', fontsize=10, fontweight='bold',
                            bbox=dict(boxstyle='round', facecolor='white', alpha=0.9))

        # X-axis labels
        for ax in [ax1, ax2, ax3]:
            ax.set_xticks([0.3, 1.3, 2.3])
            ax.set_xticklabels(['Alice', 'Bob', 'Carol'])

        if frame > 90:
            fig.text(0.5, 0.02, 'Time allocation: 43.6% Alice, 34.6% Bob, 21.8% Carol\nΣ weights = 1 (probability simplex Δ²)',
                    ha='center', fontsize=11, fontweight='bold',
                    bbox=dict(boxstyle='round', facecolor='yellow', alpha=0.8))

    anim = FuncAnimation(fig, animate, frames=140, interval=100)
    anim.save('gif6_softmax_transformation.gif', writer=PillowWriter(fps=10))
    plt.close()
    print("✓ GIF 6: Softmax transformation (scores → probabilities)")


def gif_7_value_vectors_3d():
    """
    GIF 7: Value vectors in original 3D personality space
    These are what people actually offer
    """
    viz = RomanceAttentionGeometry()

    fig = plt.figure(figsize=(14, 12))
    ax = fig.add_subplot(111, projection='3d')

    def animate(frame):
        ax.clear()

        angle = 40 + (frame % 120) * 0.6
        ax.view_init(elev=30, azim=angle)

        ax.set_xlim(0, 1.0)
        ax.set_ylim(0, 1.0)
        ax.set_zlim(0, 1.0)
        ax.set_xlabel('Humor (Actual)', fontsize=12, fontweight='bold')
        ax.set_ylabel('Intelligence (Actual)', fontsize=12, fontweight='bold')
        ax.set_zlabel('Kindness (Actual)', fontsize=12, fontweight='bold')
        ax.set_title('Value Vectors: V = X × W_V (What People Actually Offer)\n(Since W_V = I, values = original personalities)',
                    fontsize=13, fontweight='bold', pad=20)

        ax.grid(True, alpha=0.3)
        ax.scatter(0, 0, 0, color='black', s=100, marker='o')

        # Values appear sequentially
        for i in range(3):
            alpha = min((frame - i*30) / 30, 1.0) if frame >= i*30 else 0

            if alpha > 0:
                x, y, z = viz.V[i]

                arrow = Arrow3D([0, x], [0, y], [0, z],
                               mutation_scale=20, lw=4, arrowstyle='-|>',
                               color=viz.colors[i], alpha=alpha)
                ax.add_artist(arrow)

                ax.scatter(x, y, z, color=viz.colors[i], s=700,
                          edgecolors='black', linewidths=3, alpha=alpha, zorder=10)

                if alpha > 0.6:
                    ax.text(x+0.08, y, z+0.05,
                           f'v_{viz.people[i]}\n[{x:.1f}, {y:.2f}, {z:.1f}]',
                           fontsize=11, fontweight='bold', color=viz.colors[i],
                           bbox=dict(boxstyle='round', facecolor='white', alpha=0.9,
                                   edgecolor=viz.colors[i], linewidth=2))

        if frame > 90:
            ax.text2D(0.5, 0.02, 'These are the ACTUAL qualities you\'ll absorb from each person!',
                     transform=ax.transAxes, fontsize=11, ha='center', style='italic',
                     bbox=dict(boxstyle='round', facecolor='lightgreen', alpha=0.8))

    anim = FuncAnimation(fig, animate, frames=140, interval=100)
    anim.save('gif7_value_vectors_3d.gif', writer=PillowWriter(fps=10))
    plt.close()
    print("✓ GIF 7: Value vectors 3D (what people actually offer)")


def gif_8_weighted_average_blending():
    """
    GIF 8: Beautiful weighted average in 3D
    Shows attention weights pulling value vectors into output
    """
    viz = RomanceAttentionGeometry()

    fig = plt.figure(figsize=(15, 12))
    ax = fig.add_subplot(111, projection='3d')

    def animate(frame):
        ax.clear()

        angle = 35 + (frame % 140) * 0.5
        ax.view_init(elev=25, azim=angle)

        ax.set_xlim(0, 1.0)
        ax.set_ylim(0, 1.0)
        ax.set_zlim(0, 1.0)
        ax.set_xlabel('Humor', fontsize=12, fontweight='bold')
        ax.set_ylabel('Intelligence', fontsize=12, fontweight='bold')
        ax.set_zlabel('Kindness', fontsize=12, fontweight='bold')
        ax.set_title('Weighted Average: output = Σᵢ (attention_weight_i × v_i)\n(You become a blend of who you attend to)',
                    fontsize=13, fontweight='bold', pad=20)

        ax.grid(True, alpha=0.3)
        ax.scatter(0, 0, 0, color='black', s=100, marker='o')

        # Value vectors (faded)
        for i in range(3):
            x, y, z = viz.V[i]
            arrow = Arrow3D([0, x], [0, y], [0, z],
                           mutation_scale=15, lw=2, arrowstyle='-|>',
                           color=viz.colors[i], alpha=0.3, linestyle='--')
            ax.add_artist(arrow)
            ax.scatter(x, y, z, color=viz.colors[i], s=400,
                      alpha=0.3, edgecolors='black', linewidths=2)
            ax.text(x+0.05, y, z, f'v_{viz.people[i]}',
                   fontsize=9, color=viz.colors[i], alpha=0.5)

        progress = min(frame / 70, 1.0)

        # Show weighted contributions
        if progress > 0.2:
            fade = min((progress - 0.2) / 0.4, 1.0)

            for i in range(3):
                weight = viz.attention_weights[i]
                contribution = weight * viz.V[i] * fade
                x, y, z = contribution

                # Thicker arrows for contributions
                arrow_contrib = Arrow3D([0, x], [0, y], [0, z],
                                       mutation_scale=15, lw=3, arrowstyle='-|>',
                                       color=viz.colors[i], alpha=fade)
                ax.add_artist(arrow_contrib)

                # Label with percentage
                if fade > 0.6:
                    mid_x, mid_y, mid_z = x/2, y/2, z/2
                    ax.text(mid_x, mid_y, mid_z,
                           f'{weight:.1%}',
                           fontsize=10, fontweight='bold', color=viz.colors[i],
                           bbox=dict(boxstyle='round', facecolor='white', alpha=0.9))

        # Show final output
        if progress > 0.7:
            appear = min((progress - 0.7) / 0.3, 1.0)
            x, y, z = viz.output

            arrow_out = Arrow3D([0, x], [0, y], [0, z],
                               mutation_scale=20, lw=6, arrowstyle='-|>',
                               color='purple', alpha=appear)
            ax.add_artist(arrow_out)

            ax.scatter(x, y, z, color='purple', s=900, marker='*',
                      edgecolors='black', linewidths=4, alpha=appear, zorder=15)

            if appear > 0.6:
                ax.text(x+0.08, y, z+0.08,
                       f'Enriched YOU\n[{x:.2f}, {y:.2f}, {z:.2f}]',
                       fontsize=12, fontweight='bold', color='purple',
                       bbox=dict(boxstyle='round', facecolor='yellow', alpha=0.95,
                               edgecolor='purple', linewidth=3))

        if progress > 0.9:
            ax.text2D(0.5, 0.02,
                     f'You absorbed: 43.6% Alice + 34.6% Bob + 21.8% Carol\n' +
                     f'Started as [{viz.you[0]:.2f}, {viz.you[1]:.2f}, {viz.you[2]:.2f}] → ' +
                     f'Became [{viz.output[0]:.2f}, {viz.output[1]:.2f}, {viz.output[2]:.2f}]',
                     transform=ax.transAxes, fontsize=10, ha='center',
                     bbox=dict(boxstyle='round', facecolor='lightblue', alpha=0.9))

    anim = FuncAnimation(fig, animate, frames=160, interval=100)
    anim.save('gif8_weighted_average_blending.gif', writer=PillowWriter(fps=10))
    plt.close()
    print("✓ GIF 8: Weighted average blending (beautiful 3D)")


def gif_9_convex_hull_theorem():
    """
    GIF 9: The convex hull - output constrained to triangle
    Shows output cannot escape the convex hull of values
    """
    viz = RomanceAttentionGeometry()

    fig = plt.figure(figsize=(15, 12))
    ax = fig.add_subplot(111, projection='3d')

    def animate(frame):
        ax.clear()

        angle = 45 + (frame % 180) * 0.4
        ax.view_init(elev=30, azim=angle)

        ax.set_xlim(0, 1.0)
        ax.set_ylim(0.5, 1.0)
        ax.set_zlim(0.5, 0.9)
        ax.set_xlabel('Humor', fontsize=12, fontweight='bold')
        ax.set_ylabel('Intelligence', fontsize=12, fontweight='bold')
        ax.set_zlabel('Kindness', fontsize=12, fontweight='bold')
        ax.set_title('Convex Hull Theorem: output ∈ conv({v₁, v₂, v₃})\n(You can only be a blend of available influences!)',
                    fontsize=13, fontweight='bold', pad=20)

        ax.grid(True, alpha=0.3)

        # Draw convex hull (triangle)
        vertices = viz.V
        triangle = [[vertices[0], vertices[1], vertices[2]]]
        poly = Poly3DCollection(triangle, alpha=0.25, facecolor='lightblue',
                               edgecolor='blue', linewidths=3)
        ax.add_collection3d(poly)

        # Value vectors at vertices
        for i in range(3):
            x, y, z = viz.V[i]
            ax.scatter(x, y, z, color=viz.colors[i], s=700,
                      edgecolors='black', linewidths=3, alpha=0.9, zorder=10)
            ax.text(x+0.05, y, z+0.03,
                   f'v_{viz.people[i]}',
                   fontsize=11, fontweight='bold', color=viz.colors[i],
                   bbox=dict(boxstyle='round', facecolor='white', alpha=0.9))

        # Show attention weights as bars at each vertex
        progress = min(frame / 80, 1.0)

        if progress > 0.3:
            fade = min((progress - 0.3) / 0.4, 1.0)

            for i in range(3):
                x, y, z = viz.V[i]
                weight = viz.attention_weights[i]
                bar_height = weight * 0.5 * fade

                # Vertical bar showing weight
                ax.plot([x, x], [y, y], [z, z + bar_height],
                       color=viz.colors[i], linewidth=5, alpha=0.8)

                ax.scatter(x, y, z + bar_height, color=viz.colors[i], s=300,
                          edgecolors='black', linewidths=2, alpha=0.8)

                if fade > 0.6:
                    ax.text(x+0.03, y, z + bar_height + 0.02,
                           f'{weight:.1%}',
                           fontsize=10, fontweight='bold',
                           bbox=dict(boxstyle='round', facecolor='white', alpha=0.9))

        # Output appears inside triangle
        if progress > 0.7:
            appear = min((progress - 0.7) / 0.3, 1.0)
            x, y, z = viz.output

            ax.scatter(x, y, z, color='purple', s=800, marker='*',
                      edgecolors='black', linewidths=4, alpha=appear, zorder=15)

            if appear > 0.6:
                ax.text(x+0.05, y, z-0.03,
                       'output',
                       fontsize=12, fontweight='bold', color='purple',
                       bbox=dict(boxstyle='round', facecolor='yellow', alpha=0.95))

                # Lines from vertices to output
                for i in range(3):
                    v_x, v_y, v_z = viz.V[i]
                    ax.plot([v_x, x], [v_y, y], [v_z, z],
                           color=viz.colors[i], linewidth=1.5, alpha=0.4, linestyle=':')

        if progress > 0.9:
            ax.text2D(0.5, 0.95, 'Output MUST live inside the triangle!',
                     transform=ax.transAxes, fontsize=12, ha='center', fontweight='bold',
                     bbox=dict(boxstyle='round', facecolor='orange', alpha=0.8))

            ax.text2D(0.5, 0.02,
                     'Convex combination: α₁v₁ + α₂v₂ + α₃v₃ where αᵢ ≥ 0, Σαᵢ = 1\n' +
                     'Geometric constraint prevents hallucination!',
                     transform=ax.transAxes, fontsize=10, ha='center',
                     bbox=dict(boxstyle='round', facecolor='lightgreen', alpha=0.8))

    anim = FuncAnimation(fig, animate, frames=200, interval=100)
    anim.save('gif9_convex_hull_theorem.gif', writer=PillowWriter(fps=10))
    plt.close()
    print("✓ GIF 9: Convex hull theorem (3D rotating triangle)")


def gif_10_complete_attention_flow():
    """
    GIF 10: Complete flow showing all steps in sequence
    Epic compilation of the entire attention mechanism
    """
    viz = RomanceAttentionGeometry()

    fig = plt.figure(figsize=(16, 14))

    def animate(frame):
        fig.clear()

        stage = frame // 40
        progress = (frame % 40) / 40

        stages = ['Input (3D)', 'Query & Keys (2D)', 'Scores (Dot Products)',
                 'Softmax (Weights)', 'Values (3D)', 'Output (Weighted Avg)']

        fig.suptitle(f'Complete Attention Flow: Step {min(stage+1, 6)}/6 - {stages[min(stage, 5)]}\n' +
                    'From Personality Space → Linear Transformations → Similarity → Probability → Blending',
                    fontsize=14, fontweight='bold')

        if stage == 0:
            # Stage 1: Input personalities in 3D
            ax = fig.add_subplot(111, projection='3d')
            ax.view_init(elev=25, azim=30 + progress*10)
            ax.set_xlim(0, 1); ax.set_ylim(0, 1); ax.set_zlim(0, 1)
            ax.set_xlabel('Humor'); ax.set_ylabel('Intelligence'); ax.set_zlabel('Kindness')
            ax.set_title('Step 1: People in Personality Space (X ∈ ℝ³ˣ³)', fontweight='bold')

            for i in range(3):
                alpha = min((progress + i*0.3), 1.0)
                if alpha > 0:
                    x, y, z = viz.X[i]
                    ax.scatter(x, y, z, color=viz.colors[i], s=600, alpha=alpha,
                              edgecolors='black', linewidths=3)
                    if alpha > 0.5:
                        ax.text(x+0.08, y, z, viz.people[i], fontsize=10, color=viz.colors[i])

        elif stage == 1:
            # Stage 2: Query and Keys in 2D
            ax = fig.add_subplot(111, projection='3d')
            ax.view_init(elev=20, azim=35 + progress*10)
            ax.set_xlim(0, 1.2); ax.set_ylim(0, 1.5); ax.set_zlim(0, 0.5)
            ax.set_xlabel('Dim 1'); ax.set_ylabel('Dim 2')
            ax.set_title('Step 2: Project to Q, K Subspaces (W_Q, W_K)', fontweight='bold')

            # 2D plane
            xx, yy = np.meshgrid(np.linspace(0, 1.2, 10), np.linspace(0, 1.5, 10))
            zz = np.zeros_like(xx)
            ax.plot_surface(xx, yy, zz, alpha=0.1, color='lightblue')

            # Query
            q_x, q_y = viz.q
            ax.scatter(q_x, q_y, 0, color='purple', s=500, marker='*', alpha=progress)
            if progress > 0.5:
                ax.text(q_x, q_y, 0.05, 'q (YOU)', color='purple', fontweight='bold')

            # Keys
            for i in range(3):
                k_x, k_y = viz.K[i]
                ax.scatter(k_x, k_y, 0, color=viz.colors[i], s=400, alpha=progress)
                if progress > 0.5:
                    ax.text(k_x, k_y, -0.05, f'k_{viz.people[i]}', fontsize=9, color=viz.colors[i])

        elif stage == 2:
            # Stage 3: Dot product scores
            ax = fig.add_subplot(111, projection='3d')
            ax.view_init(elev=25, azim=30)
            ax.set_xlim(0, 3); ax.set_ylim(0, 1); ax.set_zlim(0, 3.5)
            ax.set_xlabel('Person'); ax.set_zlabel('Score')
            ax.set_title('Step 3: Compute Scores = q·K^T', fontweight='bold')

            for i in range(3):
                score = viz.scores[i] * progress
                ax.bar3d(i, 0, 0, 0.6, 0.6, score, color=viz.colors[i], alpha=0.7)
                if progress > 0.7:
                    ax.text(i+0.3, 0.3, score+0.1, f'{viz.scores[i]:.2f}',
                           fontsize=10, fontweight='bold', ha='center')

        elif stage == 3:
            # Stage 4: Softmax weights
            ax = fig.add_subplot(111)
            ax.set_xlim(0, 3); ax.set_ylim(0, 0.5)
            ax.set_title('Step 4: Softmax Normalization', fontweight='bold')
            ax.set_xlabel('Person'); ax.set_ylabel('Attention Weight')

            bars = ax.bar(range(3), viz.attention_weights * progress, color=viz.colors,
                         alpha=0.7, edgecolor='black', linewidth=2, width=0.6)
            ax.set_xticks(range(3))
            ax.set_xticklabels(viz.people)

            if progress > 0.6:
                for i, weight in enumerate(viz.attention_weights):
                    ax.text(i, weight * progress + 0.02, f'{weight:.1%}',
                           ha='center', fontsize=11, fontweight='bold')

        elif stage == 4:
            # Stage 5: Value vectors
            ax = fig.add_subplot(111, projection='3d')
            ax.view_init(elev=30, azim=40 + progress*10)
            ax.set_xlim(0, 1); ax.set_ylim(0, 1); ax.set_zlim(0, 1)
            ax.set_xlabel('Humor'); ax.set_ylabel('Intelligence'); ax.set_zlabel('Kindness')
            ax.set_title('Step 5: Value Vectors V = X×W_V', fontweight='bold')

            for i in range(3):
                alpha = min((progress + i*0.25), 1.0)
                if alpha > 0:
                    x, y, z = viz.V[i]
                    ax.scatter(x, y, z, color=viz.colors[i], s=600, alpha=alpha,
                              edgecolors='black', linewidths=3)
                    if alpha > 0.5:
                        ax.text(x+0.05, y, z, f'v_{viz.people[i]}', fontsize=10, color=viz.colors[i])

        else:
            # Stage 6: Final output
            ax = fig.add_subplot(111, projection='3d')
            ax.view_init(elev=28, azim=35 + progress*15)
            ax.set_xlim(0, 1); ax.set_ylim(0, 1); ax.set_zlim(0, 1)
            ax.set_xlabel('Humor'); ax.set_ylabel('Intelligence'); ax.set_zlabel('Kindness')
            ax.set_title('Step 6: Output = Σ(weight_i × v_i)', fontweight='bold')

            # Values (faded)
            for i in range(3):
                x, y, z = viz.V[i]
                ax.scatter(x, y, z, color=viz.colors[i], s=400, alpha=0.3)

            # Output
            x, y, z = viz.output
            ax.scatter(x, y, z, color='purple', s=900, marker='*',
                      edgecolors='black', linewidths=4, alpha=progress, zorder=15)

            if progress > 0.6:
                ax.text(x+0.08, y, z, f'Enriched YOU\n[{x:.2f},{y:.2f},{z:.2f}]',
                       fontsize=11, fontweight='bold', color='purple',
                       bbox=dict(boxstyle='round', facecolor='yellow', alpha=0.9))

    anim = FuncAnimation(fig, animate, frames=240, interval=120)
    anim.save('gif10_complete_attention_flow.gif', writer=PillowWriter(fps=8))
    plt.close()
    print("✓ GIF 10: Complete attention flow (6 stages, slow)")


if __name__ == "__main__":
    print("\n" + "="*70)
    print("Generating Attention Mechanism Visualizations")
    print("Romance + Rigorous Linear Algebra Edition")
    print("="*70 + "\n")

    gif_1_personality_space_3d()
    gif_2_inner_product_angle()
    gif_3_query_projection_3d_to_2d()
    gif_4_key_projections_all()
    gif_5_dot_product_scores_3d()
    gif_6_softmax_transformation()
    gif_7_value_vectors_3d()
    gif_8_weighted_average_blending()
    gif_9_convex_hull_theorem()
    gif_10_complete_attention_flow()

    print("\n" + "="*70)
    print("✅ All 10 slow-moving 3D GIFs generated successfully!")
    print("="*70)
    print("\nGenerated files:")
    for i in range(1, 11):
        print(f"  ✓ gif{i}_*.gif")
    print("\nAll visualizations:")
    print("  • Use 3D rotating views")
    print("  • Show romance context (Alice, Bob, Carol)")
    print("  • Include mathematical annotations")
    print("  • Slow frame rates (8-12 fps) for clarity")
    print("  • Match article narrative exactly")
    print("\nReady to embed in your Medium article!")
    print("="*70)


Generating Attention Mechanism Visualizations
Romance + Rigorous Linear Algebra Edition



/tmp/ipython-input-1400207331.py:164: UserWarning: Glyph 8477 (\N{DOUBLE-STRUCK CAPITAL R}) missing from font(s) Liberation Sans.
  anim.save('gif1_personality_space_3d.gif', writer=PillowWriter(fps=10))


✓ GIF 1: 3D Personality space (slow rotating, people as vectors)


/tmp/ipython-input-1400207331.py:256: UserWarning: Glyph 10216 (\N{MATHEMATICAL LEFT ANGLE BRACKET}) missing from font(s) Liberation Sans.
  anim.save('gif2_inner_product_angle.gif', writer=PillowWriter(fps=10))
/tmp/ipython-input-1400207331.py:256: UserWarning: Glyph 10217 (\N{MATHEMATICAL RIGHT ANGLE BRACKET}) missing from font(s) Liberation Sans.
  anim.save('gif2_inner_product_angle.gif', writer=PillowWriter(fps=10))


✓ GIF 2: Inner product geometry (angle θ, compatibility)


...

✓ GIF 8: Weighted average blending (beautiful 3D)


/tmp/ipython-input-1400207331.py:885: UserWarning: Glyph 8712 (\N{ELEMENT OF}) missing from font(s) Liberation Sans.
  anim.save('gif9_convex_hull_theorem.gif', writer=PillowWriter(fps=10))
/tmp/ipython-input-1400207331.py:885: UserWarning: Glyph 8321 (\N{SUBSCRIPT ONE}) missing from font(s) Liberation Sans.
  anim.save('gif9_convex_hull_theorem.gif', writer=PillowWriter(fps=10))
/tmp/ipython-input-1400207331.py:885: UserWarning: Glyph 8322 (\N{SUBSCRIPT TWO}) missing from font(s) Liberation Sans.
  anim.save('gif9_convex_hull_theorem.gif', writer=PillowWriter(fps=10))
/tmp/ipython-input-1400207331.py:885: UserWarning: Glyph 8323 (\N{SUBSCRIPT THREE}) missing from font(s) Liberation Sans.
  anim.save('gif9_convex_hull_theorem.gif', writer=PillowWriter(fps=10))
/tmp/ipython-input-1400207331.py:885: UserWarning: Glyph 7522 (\N{LATIN SUBSCRIPT SMALL LETTER I}) missing from font(s) Liberation Sans.
  anim.save('gif9_convex_hull_theorem.gif', writer=PillowWriter(fps=10))


✓ GIF 9: Convex hull theorem (3D rotating triangle)


/tmp/ipython-input-1400207331.py:1028: UserWarning: Glyph 8712 (\N{ELEMENT OF}) missing from font(s) Liberation Sans.
  anim.save('gif10_complete_attention_flow.gif', writer=PillowWriter(fps=8))
/tmp/ipython-input-1400207331.py:1028: UserWarning: Glyph 8477 (\N{DOUBLE-STRUCK CAPITAL R}) missing from font(s) Liberation Sans.
  anim.save('gif10_complete_attention_flow.gif', writer=PillowWriter(fps=8))
/tmp/ipython-input-1400207331.py:1028: UserWarning: Glyph 739 (\N{MODIFIER LETTER SMALL X}) missing from font(s) Liberation Sans.
  anim.save('gif10_complete_attention_flow.gif', writer=PillowWriter(fps=8))


✓ GIF 10: Complete attention flow (6 stages, slow)

✅ All 10 slow-moving 3D GIFs generated successfully!

Generated files:
  ✓ gif1_*.gif
  ✓ gif2_*.gif
  ✓ gif3_*.gif
  ✓ gif4_*.gif
  ✓ gif5_*.gif
  ✓ gif6_*.gif
  ✓ gif7_*.gif
  ✓ gif8_*.gif
  ✓ gif9_*.gif
  ✓ gif10_*.gif

All visualizations:
  • Use 3D rotating views
  • Show romance context (Alice, Bob, Carol)
  • Include mathematical annotations
  • Slow frame rates (8-12 fps) for clarity
  • Match article narrative exactly

Ready to embed in your Medium article!
